### 0 Import Libraries

In [4]:
import os
import rarfile
import pandas as pd
from tqdm import tqdm

### 1 Extraction of Data from RAR Files

In [5]:
def extract_file(rar_file, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    with rarfile.RarFile(rar_file) as rf:
        for f in tqdm(rf.infolist(), desc='Extracting files', unit='file'):
            rf.extract(f, output_dir)

os.makedirs('raw-data', exist_ok=True)

In [ ]:
# Create a folder named 'renamed-data' to store the renamed files
if not os.path.exists('renamed-data'):
    os.makedirs('renamed-data')

provinces = ['Gilan', 'Golestan', 'Mazandaran']

for province in provinces:
    # Create folders Hourly and Daily
    if not os.path.exists('renamed-data/' + province):
        os.makedirs('renamed-data/' + province)
    if not os.path.exists('renamed-data/' + province + '/Hourly'):
        os.makedirs('renamed-data/' + province + '/Hourly')
    if not os.path.exists('renamed-data/' + province + '/Daily'):
        os.makedirs('renamed-data/' + province + '/Daily')

In [6]:
# Loop through all the files in the compressed-data folder and extract .rar files
for file in tqdm(os.listdir('compressed-data')):
    if file.endswith('.rar'):
        extract_file(os.path.join('compressed-data', file), f'raw-data/{file.replace(".rar", "")}')

100%|██████████| 44/44 [00:27<00:00,  1.59it/s]


### 2 Accumulation of Data

In [7]:
# Create a folder named accumulated-data
os.makedirs('accumulated-data', exist_ok=True)

names_english = ['Gilan', 'Mazandaran', 'Golestan']
names_persian = ['گيلان', 'مازندران', 'گلستان']

# In the accumulated-data folder, create a folder for each province
for name in names_english:
    os.makedirs(f'accumulated-data/{name}', exist_ok=True)

for folder in tqdm(os.listdir('raw-data')):
    # If the folder name contains the name of a province in Persian or English case-insensitively
    # Move the folder to the corresponding province folder in the accumulated-data folder
    for i in range(3):
        if names_english[i].lower() in folder.lower() or names_persian[i] in folder:
            os.rename(f'raw-data/{folder}', f'accumulated-data/{names_english[i]}/{folder}')

# If raw-data folder is empty, remove it
if not os.listdir('raw-data'):
    os.rmdir('raw-data')

100%|██████████| 44/44 [00:00<00:00, 8505.76it/s]


In [8]:
# Create a folder named converted data
os.makedirs('converted-data', exist_ok=True)

# Loop through all the folders in the accumulated-data folder
for folder in tqdm(os.listdir('accumulated-data')):
    # For each folder, create a folder with the same name in the converted-data folder
    os.makedirs(f'converted-data/{folder}', exist_ok=True)
    # Create a Daily and Hourly folder inside the folder
    os.makedirs(f'converted-data/{folder}/Daily', exist_ok=True)
    os.makedirs(f'converted-data/{folder}/Hourly', exist_ok=True)

    for root, _, files in os.walk(f'accumulated-data/{folder}'):
        for file in files:
            # If the file is an excel file
            if file.endswith('.xlsx'):
                # Read the excel file
                df = pd.read_excel(os.path.join(root, file))
                # Set the first row as the column names
                df.columns = df.iloc[0]
                df = df.drop(0)
                # If the name contains 'daily' case-insensitively
                if 'daily' in file.lower():
                    # Check if there is file with the same name in the Daily folder
                    if os.path.exists(f'converted-data/{folder}/Daily/{file.replace(".xlsx", ".csv")}'):
                        # Read the file in the Hourly folder
                        df_hourly = pd.read_csv(f'converted-data/{folder}/Daily/{file.replace(".xlsx", ".csv")}')
                        # Concatenate the two dataframes
                        df = pd.concat([df_hourly, df], ignore_index=True)
                    # Save the file in the Daily folder
                    df.to_csv(f'converted-data/{folder}/Daily/{file.replace(".xlsx", ".csv")}', index=False)
                # If the name contains 'hourly' case-insensitively
                elif 'hourly' in file.lower():
                    # Check if there is file with the same name in the Hourly folder
                    if os.path.exists(f'converted-data/{folder}/Hourly/{file.replace(".xlsx", ".csv")}'):
                        # Read the file in the Hourly folder
                        df_hourly = pd.read_csv(f'converted-data/{folder}/Hourly/{file.replace(".xlsx", ".csv")}')
                        # Concatenate the two dataframes
                        df = pd.concat([df_hourly, df], ignore_index=True)
                    # Save the file in the Hourly folder
                    df.to_csv(f'converted-data/{folder}/Hourly/{file.replace(".xlsx", ".csv")}', index=False)

100%|██████████| 3/3 [03:53<00:00, 77.99s/it]
